# Preprocess Data Of Condition 1, 2 and 3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import scipy.stats as stats
from scipy.stats import pearsonr
import random
import datetime
import scipy
import math

from params import *
from utils import *
from key import *

/opt/anaconda3/envs/web_development/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password=ssh_password,
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd=db_password,
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        df_1 = pd.read_sql("SELECT * FROM condition_1;", connection)
        df_1["experiment_type"] = "condition_1"
        df_2 = pd.read_sql("SELECT * FROM condition_2;", connection)
        df_2["experiment_type"] = "condition_2"
        df_3 = pd.read_sql("SELECT * FROM condition_3;", connection)
        df_3["experiment_type"] = "condition_3"
        print(f"Data read finished, length: condition 1 length {len(df_1)}, condition 2 length {len(df_2)}, condition 3 length {len(df_3)}")
    
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

df_original = pd.concat([df_1, df_2, df_3])
df_original.tail(5)

2024-03-16 21:48:36,936| ERROR   | Password is required for key /Users/guodawei/.ssh/id_rsa


Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished, length: condition 1 length 297, condition 2 length 600, condition 3 length 297


,participantId,assignmentId,projectId,attention_passed,total_time,identity_choices,ideologies,bot_detected,submit_time,non_ideology_answers,additional_answers,reason,experiment_type,ideology_answers,labels
292,FA68BF716AA445A384E204CF28D6CC6D,3ed58875-f0ef-4017-9cda-1c1aabcb0f4a,2da4e70d-29e6-4eaa-bff5-d9d7de00da7a,1,763.812,"[[6, 5], [4, 0], [8, 3]]",None,50,2023-10-12 08:29:52,"[{""answers"": [-100, 1.1333333333333329, -100],...","[[1.1, 1.5, -0.9]]",,condition_3,"[{""answers"": [1, 1, 0], ""time_to_answer"": [6.3...","[[2], [2], [0]]"
293,FC0A2CD7495F49A68934E3AF7939572F,5db466be-b506-4ca2-8c3b-ccbd376cf09f,2da4e70d-29e6-4eaa-bff5-d9d7de00da7a,1,788.396,"[[1, 3], [15, 2], [19, 5]]",None,35,2023-10-12 06:15:37,"[{""answers"": [-100, 3, -100], ""time_to_answer""...","[[-1.9, -1.5, 2]]",The consistent reaction speed seemed like a bo...,condition_3,"[{""answers"": [1, 0, 0], ""time_to_answer"": [5.7...","[[2, 9, 5], [0, 8, 6], [0, 4, 8]]"
294,FD264C6FCF6E42D784A7735447BC9618,53ec3702-4a45-424f-a539-3440dc2d28b3,2da4e70d-29e6-4eaa-bff5-d9d7de00da7a,1,553.751,"[[14, 5], [16, 2], [6, 3]]",None,52,2023-10-12 06:15:07,"[{""answers"": [-100, 0.1, -100], ""time_to_answe...","[[2, -2, -2]]",,condition_3,"[{""answers"": [1, 1, 0], ""time_to_answer"": [7.6...","[[2, 7], [0, 5, 8], [0, 4, 6]]"
295,FDE676393D42487380BB9E9FF2561795,b5abb789-50db-4597-8ce6-60505d0ed814,2da4e70d-29e6-4eaa-bff5-d9d7de00da7a,1,1618.780,"[[9, 2], [5, 3], [15, 1]]",None,13,2023-10-12 06:13:51,"[{""answers"": [-100, 0.9333333333333336, -100],...","[[0.9, 2, 1]]",NA,condition_3,"[{""answers"": [0, 1, 0], ""time_to_answer"": [7.6...","[[4, 8, 2], [5, 2, 7], [2, 8, 6]]"
296,FE415398E9F841EB88B0D293CC10F777,23205573-1f84-4655-a50b-c5d3538e983a,2da4e70d-29e6-4eaa-bff5-d9d7de00da7a,1,430.702,"[[16, 4], [2, 1], [14, 0]]",None,52,2023-09-17 15:26:23,"[{""answers"": [-100, 1.7, -100], ""time_to_answe...","[[1.7, -2, -1.9]]",I didn't choose bot because I thought it was s...,condition_3,"[{""answers"": [1, 0, 0], ""time_to_answer"": [8.4...","[[2, 7, 9], [4, 6, 8], [4, 6, 8]]"


In [3]:
df = df_original.copy()

### 1.1 Attention check

In [10]:
failed_attention_ids = df.loc[df["attention_passed"] == 0]["participantId"].tolist()
print(f"{len(failed_attention_ids) } participants failed the attention check: {failed_attention_ids}")
df = df.loc[df["attention_passed"] == 1]
columns_to_drop = ['assignmentId', 'projectId', 'attention_passed', 'total_time', 'identity_choices', 'ideologies', 'submit_time', "reason"]
df.drop(columns=columns_to_drop, inplace=True, axis=1)

cols_to_parse = ["non_ideology_answers", "additional_answers", "labels"]
for col in cols_to_parse:
    df[col] = df[col].apply(lambda x: literal_eval(x) if pd.notnull(x) else x)
df.set_index("participantId", inplace=True)

# Data "ideology_answers" of some participants are corrupted, so we need to remove theme
data_error_ids = []
for id, row in df.iterrows():
    if pd.notnull(row["ideology_answers"]):
        try:
            literal_eval(row["ideology_answers"])
        except:
            data_error_ids.append(id)
            df.drop(index=id, inplace=True, axis=0)
print(f"There are {len(data_error_ids)} data errors, ids: {data_error_ids}")
df.drop(columns=["ideology_answers"], inplace=True, axis=1)
df.tail(3)

32 participants failed the attention check: ['363DEB2413FB48FD9D4CDCFD0EFE8171', '37A77F60BA16417D9170FF3AB76FFED3', '5911B1A9AE6D4E3388B295D078B7F49A', '60B62D2058D24418965396147B2746C3', '85364190604C4998AE9892A92449CB28', '9B32FED49EA64ABF85DB84B245455B26', 'DAF3701025BB4377BC901C1D6989C6A6', 'F5F966E8A63F467E9ABC254186478292', '0660BC35F9D84A76AD94D044AB3E2F03', '07CB55A4B0EF4F969562F92636F194C4', '0AAAF5A90BE94134BA67E17D5C757684', '2E5CC51E01154406807CDCAB36006E62', '5CEE6A3A286F4719B90C4676FCD8E2FC', '67D7C1011797450CA613E3C92A47118F', '67E7D55F665A49D38C82E49C72B7077B', '69E4C35A0296446D87A626AF9CDA1A71', '755DFFC11EE74976929C37FF43133426', '79AF6A78556249CBAE9F5E92BD9A10A1', '7C7C5A02D261468B933EF149B2A04D9D', '8CBEF3832F99441C89B7952F00421DC6', '9FF5D3BF714E4790AA2D2CEE3F99C74E', 'ADD051D15E114D4CB67AD31B0FA8F673', 'B9E4A5C4C1D6458F9DEF6E9C7D4CB92F', '0B73691B78694428A89DC54000E21248', '208C4E3596D44FCF81020892DBFF3DDA', '442A4DBB859746FE8F9EDBEB6AD1A519', '54595FC00C524A20A8

,bot_detected,non_ideology_answers,additional_answers,experiment_type,labels
participantId,,,,,
FC0A2CD7495F49A68934E3AF7939572F,35,"[{'answers': [-100, 3, -100], 'time_to_answer'...","[[-1.9, -1.5, 2]]",condition_3,"[[2, 9, 5], [0, 8, 6], [0, 4, 8]]"
FD264C6FCF6E42D784A7735447BC9618,52,"[{'answers': [-100, 0.1, -100], 'time_to_answe...","[[2, -2, -2]]",condition_3,"[[2, 7], [0, 5, 8], [0, 4, 6]]"
FDE676393D42487380BB9E9FF2561795,13,"[{'answers': [-100, 0.9333333333333336, -100],...","[[0.9, 2, 1]]",condition_3,"[[4, 8, 2], [5, 2, 7], [2, 8, 6]]"


### 1.2. Calculate equivalent ideology from labels

Labels mapping:
- **None**: No ideology-related labels are chosen for that participant. These values aren't included in further analysis.
- **0**: Both a label indicating a liberal ideology (*Liberal*, *Somewhat liberal*), and a label indicating a conservative ideology (*Conservative*, *Somwhat conservative*) are chosen.
- **1/2**: If *Conservative* is chosen, then the equivalent value is 2. If only *Somewhat conservative* is chosen, then the value is 1.
- **-1/-2**: Similar as above.

In [11]:
df["labels"] = df["labels"].apply(lambda x: x if type(x) == list else [None, None, None])

def ideology_from_labels(labels):
    if labels is None:
        return None
    
    if set(labels).intersection({0, 1, 2, 3}) == set():
        return None
    
    if set(labels).intersection({0, 1}) != set() and set(labels).intersection({2, 3}) != set():
        return 0
    
    ideology = 0
    if 3 in labels:
        ideology = 1
    if 2 in labels:
        ideology = 2
    if 1 in labels:
        ideology = -1
    if 0 in labels:
        ideology = -2
    return ideology

df["ideology_from_labels"] = df["labels"].apply(lambda labels_list: [ideology_from_labels(labels) for labels in labels_list])
df.tail() 

,bot_detected,non_ideology_answers,additional_answers,experiment_type,labels,ideology_from_labels
participantId,,,,,,
FA1AD18335DE4D4CAB7F87C576F4A762,51,"[{'answers': [-100, 1.6, -100], 'time_to_answe...","[[2, -1.1, -0.4]]",condition_3,"[[2, 7], [4, 1], [1, 4]]","[2, -1, -1]"
FA68BF716AA445A384E204CF28D6CC6D,50,"[{'answers': [-100, 1.1333333333333329, -100],...","[[1.1, 1.5, -0.9]]",condition_3,"[[2], [2], [0]]","[2, 2, -2]"
FC0A2CD7495F49A68934E3AF7939572F,35,"[{'answers': [-100, 3, -100], 'time_to_answer'...","[[-1.9, -1.5, 2]]",condition_3,"[[2, 9, 5], [0, 8, 6], [0, 4, 8]]","[2, -2, -2]"
FD264C6FCF6E42D784A7735447BC9618,52,"[{'answers': [-100, 0.1, -100], 'time_to_answe...","[[2, -2, -2]]",condition_3,"[[2, 7], [0, 5, 8], [0, 4, 6]]","[2, -2, -2]"
FDE676393D42487380BB9E9FF2561795,13,"[{'answers': [-100, 0.9333333333333336, -100],...","[[0.9, 2, 1]]",condition_3,"[[4, 8, 2], [5, 2, 7], [2, 8, 6]]","[2, 2, 2]"


### 1.3. Generate a row for each question

In [12]:
df["bot_detected"] = df["bot_detected"].apply(lambda x: [(x // 10) == 3, None, (x % 10) == 3])
df["additional_answers"] = df["additional_answers"].apply(lambda x: x[0])
df = df.explode("non_ideology_answers")
for col in ["answers", "idx_of_question", "who_answers_first"]:
    df[col] = df["non_ideology_answers"].apply(lambda x: x[col])
df["who"] = df.apply(lambda x: [0, 1, 2], axis=1)
df["first_answer"] = df.apply(lambda row: row["answers"][row["who_answers_first"]], axis=1)
df["ideology_all_zeros"] = df["additional_answers"].apply(lambda x: x[0] == 0 and x[1] == 0 and x[2] == 0)
df["first_ideology"] = df.apply(lambda row: row["additional_answers"][row["who_answers_first"]], axis=1)
df["human_answer"] = df["answers"].apply(lambda x: x[1])
df["human_ideology"] = df["additional_answers"].apply(lambda x: x[1])
df["human_ideology_from_labels"] = df["ideology_from_labels"].apply(lambda x: x[1])
df = df.explode(["bot_detected", "additional_answers", "answers", "who", "labels", "ideology_from_labels"])
df["ideology"] = df["additional_answers"]
df["answer"] = df["answers"]
df = df.loc[:, ["experiment_type", "idx_of_question", "who", "who_answers_first", "answer", "ideology", "ideology_from_labels", "first_answer", "first_ideology", "human_answer", "human_ideology", "human_ideology_from_labels", "bot_detected", "ideology_all_zeros"]]
df.tail()

,experiment_type,idx_of_question,who,who_answers_first,answer,ideology,ideology_from_labels,first_answer,first_ideology,human_answer,human_ideology,human_ideology_from_labels,bot_detected,ideology_all_zeros
participantId,,,,,,,,,,,,,,
FDE676393D42487380BB9E9FF2561795,condition_3,5,1,1,1.1,2,2,1.1,2.0,1.1,2.0,2.0,None,False
FDE676393D42487380BB9E9FF2561795,condition_3,5,2,1,-100,1,2,1.1,2.0,1.1,2.0,2.0,True,False
FDE676393D42487380BB9E9FF2561795,condition_3,4,0,2,-100,0.9,2,1.1,1.0,1.9,2.0,2.0,False,False
FDE676393D42487380BB9E9FF2561795,condition_3,4,1,2,1.9,2,2,1.1,1.0,1.9,2.0,2.0,None,False
FDE676393D42487380BB9E9FF2561795,condition_3,4,2,2,1.1,1,2,1.1,1.0,1.9,2.0,2.0,True,False


### 1.4. Generate random values for bots' answers when human participant answers first

In [13]:
import random
random.seed(40)     # set a random seed to ensure reproducibility
df["answer"] = df["answer"].apply(lambda answer: random.uniform(-3, 3) if answer == -100 else answer)
df.head()

,experiment_type,idx_of_question,who,who_answers_first,answer,ideology,ideology_from_labels,first_answer,first_ideology,human_answer,human_ideology,human_ideology_from_labels,bot_detected,ideology_all_zeros
participantId,,,,,,,,,,,,,,
000D1F259F974E9682C4EBD24445552A,condition_1,3,0,1,-0.248360,0,None,3.0,2.0,3.0,2.0,NaN,True,False
000D1F259F974E9682C4EBD24445552A,condition_1,3,1,1,3.000000,2,None,3.0,2.0,3.0,2.0,NaN,None,False
000D1F259F974E9682C4EBD24445552A,condition_1,3,2,1,2.267207,0,None,3.0,2.0,3.0,2.0,NaN,True,False
000D1F259F974E9682C4EBD24445552A,condition_1,0,0,0,-2.800000,0,None,-2.8,0.0,3.0,2.0,NaN,True,False
000D1F259F974E9682C4EBD24445552A,condition_1,0,1,0,3.000000,2,None,-2.8,0.0,3.0,2.0,NaN,None,False


In [14]:
df.to_csv("data/condition_data.csv")